In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# Cargar datasets
df_orders = pd.read_excel('./datasets/df_orders.xlsx')
df_vehicle = pd.read_excel('./datasets/df_vehicle.xlsx')
df_distance_km = pd.read_excel('./datasets/df_distance_km.xlsx')

In [3]:
# Preprocesar datos
df_orders[["mes", "año"]] = df_orders["mes_anio"].str.split("-", expand=True).astype(int)
df_orders.drop(columns=["mes_anio"], inplace=True)
df_orders["cliente"] = df_orders["cliente"].apply(lambda x: f'cliente_{x}')

In [4]:
class GeneticAlgorithm:
    def __init__(self, num_generations, population_size, mutation_rate):
        self.num_generations = num_generations
        self.population_size = population_size
        self.mutation_rate = mutation_rate

    def initialize_population(self, num_clients, num_vehicles, demands, vehicle_capacities):
        population = []
        while len(population) < self.population_size:
            individual = {v: [] for v in range(num_vehicles)}
            clients = list(range(1, num_clients + 1))  # Clientes del 1 al num_clients
            random.shuffle(clients)
            for client in clients:
                vehicle = random.choice(range(num_vehicles))
                individual[vehicle].append(client)
            if self.validate_individual(individual, demands, vehicle_capacities):
                population.append(individual)
        return population

    def validate_individual(self, individual, demands, vehicle_capacities):
        for v, route in individual.items():
            if sum(demands[client] for client in route) > vehicle_capacities[v]:
                return False
        return True

    def evaluate_fitness(self, individual, distance_matrix, demands, vehicle_capacities, vehicle_costs):
        total_cost = 0
        client_assignment = {}

        for v, route in individual.items():
            if not route:
                continue
            route_demand = sum(demands[client] for client in route)
            if route_demand > vehicle_capacities[v]:
                return float('inf')  # Penalización por exceder la capacidad
            route_distance = distance_matrix[0][route[0]]  # Almacén al primer cliente
            for i in range(len(route) - 1):
                route_distance += distance_matrix[route[i]][route[i + 1]]
            route_distance += distance_matrix[route[-1]][0]  # Último cliente al almacén
            route_cost = route_distance * vehicle_costs[v]
            total_cost += route_cost

            for client in route:
                if client not in client_assignment or route_cost < client_assignment[client][1]:
                    client_assignment[client] = (v, route_cost)

        unique_routes = {v: [] for v in individual.keys()}
        for client, (vehicle, _) in client_assignment.items():
            unique_routes[vehicle].append(client)

        individual.clear()
        individual.update(unique_routes)

        return total_cost

    def select_parents(self, population, fitness_scores):
        valid_population = [(ind, 1 / f) for ind, f in zip(population, fitness_scores) if f != float('inf')]
        if not valid_population:
            raise ValueError("No valid individuals found in the population.")
        individuals, weights = zip(*valid_population)
        total_fitness = sum(weights)
        probabilities = [w / total_fitness for w in weights]
        parent1 = random.choices(individuals, probabilities)[0]
        parent2 = random.choices(individuals, probabilities)[0]
        return parent1, parent2

    def crossover(self, parent1, parent2):
        child = {v: [] for v in parent1.keys()}
        for v in parent1.keys():
            crossover_point = random.randint(0, len(parent1[v]))
            child[v] = parent1[v][:crossover_point] + [
                client for client in parent2[v] if client not in parent1[v][:crossover_point]
            ]
        return child

    def mutate(self, individual):
        for v in individual.keys():
            if random.random() < self.mutation_rate and len(individual[v]) >= 2:
                i, j = random.sample(range(len(individual[v])), 2)
                individual[v][i], individual[v][j] = individual[v][j], individual[v][i]

    def run(self, distance_matrix, demands, vehicle_capacities, vehicle_costs):
        num_clients = len(demands) - 1
        num_vehicles = len(vehicle_capacities)

        population = self.initialize_population(num_clients, num_vehicles, demands, vehicle_capacities)

        for generation in range(self.num_generations):
            fitness_scores = [
                self.evaluate_fitness(ind, distance_matrix, demands, vehicle_capacities, vehicle_costs)
                for ind in population
            ]
            new_population = []
            for _ in range(self.population_size):
                parent1, parent2 = self.select_parents(population, fitness_scores)
                child = self.crossover(parent1, parent2)
                self.mutate(child)
                new_population.append(child)
            population = new_population

        fitness_scores = [
            self.evaluate_fitness(ind, distance_matrix, demands, vehicle_capacities, vehicle_costs)
            for ind in population
        ]
        best_index = np.argmin(fitness_scores)
        best_solution = population[best_index]
        best_cost = fitness_scores[best_index]

        return best_solution, best_cost


# Preparar datos
distance_matrix = df_distance_km.to_numpy()
demands = np.append(df_orders['order_demand'].to_numpy(), 0)  # Agregar demanda 0 para el almacén
vehicle_capacities = df_vehicle['capacidad_kg'].to_numpy()
vehicle_costs = df_vehicle['costo_km'].to_numpy()

# Configuración
ga = GeneticAlgorithm(num_generations=100, population_size=50, mutation_rate=0.1)
best_solution, best_cost = ga.run(distance_matrix, demands, vehicle_capacities, vehicle_costs)

# Mostrar resultados
print("Mejor solución (incluyendo regreso al almacén):")
vehicle_costs_detail = {}
for vehicle, route in best_solution.items():
    if route:  # Si la ruta no está vacía
        route_with_depot = [0] + route + [0]  # Añadir almacén al inicio y al final
        route_distance = (
            sum(distance_matrix[route_with_depot[i]][route_with_depot[i + 1]] for i in range(len(route_with_depot) - 1))
        )
        vehicle_cost = route_distance * vehicle_costs[vehicle]
        vehicle_costs_detail[vehicle] = vehicle_cost
        print(f"Vehículo {vehicle}: Ruta -> {route_with_depot}")
        print(f"  Coste del vehículo: {vehicle_cost:.2f} €")
    else:  # Si la ruta está vacía
        vehicle_costs_detail[vehicle] = 0
        print(f"Vehículo {vehicle}: Ruta -> [0]")
        print(f"  Coste del vehículo: 0.00 €")

print("\nCostes por vehículo:")
for vehicle, cost in vehicle_costs_detail.items():
    print(f"  Vehículo {vehicle}: {cost:.2f} €")



Mejor solución (incluyendo regreso al almacén):
Vehículo 0: Ruta -> [0, 1, 9, 0]
  Coste del vehículo: 3.60 €
Vehículo 1: Ruta -> [0, 11, 20, 8, 3, 6, 0]
  Coste del vehículo: 5.10 €
Vehículo 2: Ruta -> [0, 4, 7, 17, 16, 13, 0]
  Coste del vehículo: 6.15 €
Vehículo 3: Ruta -> [0, 18, 2, 0]
  Coste del vehículo: 8.32 €
Vehículo 4: Ruta -> [0, 12, 15, 10, 0]
  Coste del vehículo: 4.90 €
Vehículo 5: Ruta -> [0, 19, 5, 14, 0]
  Coste del vehículo: 9.49 €

Costes por vehículo:
  Vehículo 0: 3.60 €
  Vehículo 1: 5.10 €
  Vehículo 2: 6.15 €
  Vehículo 3: 8.32 €
  Vehículo 4: 4.90 €
  Vehículo 5: 9.49 €
